In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, when, isnan, countDistinct,sum
import pyspark.sql.functions as F
import pandas as pd

In [2]:
spark = SparkSession.builder.appName("LoanDataQuality").getOrCreate()

In [4]:
df_loans = spark.read.option('header','true').csv("LCloan_raw_data.csv")

In [5]:
from PySpark_testing import *
from PySpark_testing import check_consistency

In [6]:
full_mapping = pd.read_csv("full_mapping.csv")

In [17]:
focused_mapping= pd.read_csv('focused_mapping.csv')

In [19]:
filtered_df = focused_mapping[focused_mapping['test_function'].apply(lambda x: 'validate_date_format' in x)]
print(filtered_df['column_name'])

1            acc_open_past_24mths
10       chargeoff_within_12_mths
12     collections_12_mths_ex_med
40           mo_sin_old_rev_tl_op
52                   next_pymnt_d
63               num_tl_120dpd_2m
64                   num_tl_30dpd
65             num_tl_90g_dpd_24m
66             num_tl_op_past_12m
68                    open_acc_6m
72                    open_rv_12m
73                    open_rv_24m
122               hardship_status
124               hardship_amount
125           hardship_start_date
126             hardship_end_date
127       payment_plan_start_date
128               hardship_length
129                  hardship_dpd
138               settlement_date
141               settlement_term
146      chargeoff_within_12_mths
147    collections_12_mths_ex_med
158          mo_sin_old_rev_tl_op
173              num_tl_120dpd_2m
174                  num_tl_30dpd
175            num_tl_90g_dpd_24m
176            num_tl_op_past_12m
177                   open_acc_6m
181           

In [21]:
df2 = spark.read.csv('new.csv', header=True, inferSchema=True)

In [52]:
#df2.columns

### functions validations before running


In [26]:
check_consistency(df_loans,df2, column='acc_open_past_24mths', key='addr_state')


(False, 'Number of mismatched rows: 580119')

In [ ]:
check_foreign_key(df_loans, 'acc_now_delinq', df2, 'addr_state')

In [36]:
dtype_consistency(df_loans,'addr_state', 'string') 

All values in column 'addr_state' are consistent with the expected data type string.


(True, 0)

In [50]:
validate_numeric_range(df_loans, 'funded_amnt', 2290, 7000)

(False,
 "1826460 values in 'funded_amnt' are out of range. Values below minimum (2290): 50143,values above maximum (7000): 1776317.")

In [ ]:
validate_values_length(df,column_name, data_type)

In [ ]:
validate_allowed_values(df: DataFrame, column_name: str, allowed_values: list)

In [ ]:
file_timeliness(file_path, hours_threshold=24)

In [ ]:
 validate_date_range(df, column, duration, time_unit)

In [ ]:
moving_average(df, column, partitioned_column,date_column, tolerance_deviation, window_size):

Column<'funded_amnt[0]'>

### creating the operational table

In [23]:
operational_table = focused_mapping.copy()

In [25]:
# all the tests will be enabled by default at first
operational_table['enabled'] = True

In [29]:
# logic for all validation test type to be critical 
operational_table['critical'] = operational_table['test_type'].apply(
    lambda types: 'validation' in types)

In [31]:
exploded = operational_table.explode('test_function').reset_index(drop=True)

In [33]:
final_cols = [
    'column_name',
    'column_type',
    'business_category',
    'test_function',
    'test_type',
    'enabled',
    'critical'
]


operational_table = exploded[final_cols]

In [35]:
operational_table.to_csv('operational_table.csv', index=False)